In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime as dt
import random
from sportsreference.ncaab.teams import Teams
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from matplotlib import pyplot

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
teams_df = pd.DataFrame()
i = 2010
while i <=2018:
    print("Downloading data for the", i, "season.")
    start = dt.datetime.now()
    teams = Teams(year = i)
    end = dt.datetime.now()
    teams_temp = teams.dataframes
    teams_temp['Season'] = i
    teams_df = pd.concat([teams_df, teams_temp])
    span = end - start
    print(i, "took", span, "seconds to download.")
    i += 1

2010 took 0:00:28.156011 seconds to download.
2011 took 0:00:27.404459 seconds to download.
2012 took 0:00:27.553086 seconds to download.
2013 took 0:00:31.877854 seconds to download.
2014 took 0:00:33.598557 seconds to download.
2015 took 0:00:27.984510 seconds to download.
2016 took 0:00:29.177386 seconds to download.
2017 took 0:00:27.918015 seconds to download.
2018 took 0:00:29.590073 seconds to download.


In [3]:
games = pd.read_csv(r'data/TourneyCompactResults.csv')

In [4]:
games = games[['Season','WTeamID','LTeamID']]
games['Winner'] = games['WTeamID']
games = games.rename(columns = {'WTeamID' : 'Team_A', 'LTeamID': 'Team_B'})
games.head()

,Season,Team_A,Team_B,Winner
0,1985,1116,1234,1116
1,1985,1120,1345,1120
2,1985,1207,1250,1207
3,1985,1229,1425,1229
4,1985,1242,1325,1242


In [5]:
i = 0
games['Team_X'] = 0
games['Team_Y'] = 0
while i < len(games):
    games['Team_X'][i] = random.choice(([games['Team_A'][i],games['Team_B'][i]]))
    i += 1
games['Team_Y'] = np.where(games['Team_X'] == games['Team_A'], games['Team_B'], games['Team_A'])
games['Result'] = np.where(games['Winner'] == games['Team_X'], 'X', 'Y')
games = games.drop(columns = ['Team_A','Team_B'], axis = 1)

In [6]:
team_data = teams_df[['Season','abbreviation','assist_percentage','block_percentage','effective_field_goal_percentage','field_goal_percentage','free_throw_attempt_rate','free_throw_percentage','net_rating','offensive_rating','offensive_rebound_percentage','pace','simple_rating_system','steal_percentage','strength_of_schedule','three_point_attempt_rate','three_point_field_goal_percentage','total_rebound_percentage','true_shooting_percentage','turnover_percentage','two_point_field_goal_percentage','win_percentage']]

In [7]:
games = games.loc[games['Season'] >= 2010]

In [8]:
team_abb = pd.read_csv(r'data/team_names.csv')

In [9]:
team_abb = team_abb[['abbreviation','Team_Id']]

In [10]:
team_data_abb = team_data.merge(team_abb, left_on = 'abbreviation', right_on = 'abbreviation', how = 'left').dropna()
team_data_abb['Team_Id'] = team_data_abb['Team_Id'].astype(np.int64)

In [11]:
games_a = games.merge(team_data_abb, left_on = ['Team_X','Season'], right_on = ['Team_Id','Season'], how = 'left')
games_b = games_a.merge(team_data_abb, left_on = ['Team_Y','Season'], right_on = ['Team_Id','Season'], how = 'left')
games_b = games_b.dropna()

In [12]:
games_b['Winner'] = np.where(games_b['Winner'] == games_b['Team_X'], games_b['abbreviation_x'], games_b['abbreviation_y'])
games_b['Team_X'] = games_b['abbreviation_x']
games_b['Team_Y'] = games_b['abbreviation_y']

In [13]:
ml_input = games_b.drop(columns = ['Season','Winner','Team_X','Team_Y','abbreviation_x','abbreviation_y','Team_Id_x','Team_Id_y','win_percentage_x','win_percentage_y'], axis = 1)

In [95]:
X_all = ml_input.drop(['Result'],1)
y_all = ml_input['Result']

In [96]:
cols = [['assist_percentage_x','block_percentage_x','effective_field_goal_percentage_x','field_goal_percentage_x','free_throw_attempt_rate_x','free_throw_percentage_x','net_rating_x','offensive_rating_x','offensive_rebound_percentage_x','pace_x','simple_rating_system_x','steal_percentage_x','strength_of_schedule_x','three_point_attempt_rate_x','three_point_field_goal_percentage_x','total_rebound_percentage_x','true_shooting_percentage_x','turnover_percentage_x','two_point_field_goal_percentage_x','assist_percentage_y','block_percentage_y','effective_field_goal_percentage_y','field_goal_percentage_y','free_throw_attempt_rate_y','free_throw_percentage_y','net_rating_y','offensive_rating_y','offensive_rebound_percentage_y','pace_y','simple_rating_system_y','steal_percentage_y','strength_of_schedule_y','three_point_attempt_rate_y','three_point_field_goal_percentage_y','total_rebound_percentage_y','true_shooting_percentage_y','turnover_percentage_y','two_point_field_goal_percentage_y']]
for col in cols:
    X_all[col] = scale(X_all[col])

In [97]:
len(X_all)

533

In [98]:
#for measuring training time
from time import time 
# F1 score (also F-score or F-measure) is a measure of a test's accuracy. 
#It considers both the precision p and the recall r of the test to compute 
#the score: p is the number of correct positive results divided by the number of 
#all positive results, and r is the number of correct positive results divided by 
#the number of positive results that should have been returned. The F1 score can be 
#interpreted as a weighted average of the precision and recall, where an F1 score 
#reaches its best value at 1 and worst at 0.
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    time_taken = end - start
    
    # Print the results
    print('Trained model', clf.__class__.__name__, 'in', time_taken, ' seconds')

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    # Print and return results
    time_taken = end - start
    print("Made predictions in", time_taken, "seconds.")
    
    return f1_score(target, y_pred, pos_label='X'), sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    #print("Training something using a training set size of %d. . .").format(len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print("F1 score and accuracy score for training set:", f1, " , ", acc)
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set:", f1, " , ", acc)

In [99]:
teams_19 = Teams(year = 2019)
teams_df_19 = teams.dataframes
teams_df_19 = teams_df_19[['abbreviation','assist_percentage','block_percentage','effective_field_goal_percentage','field_goal_percentage','free_throw_attempt_rate','free_throw_percentage','free_throws_per_field_goal_attempt','net_rating','offensive_rating','offensive_rebound_percentage','opp_assist_percentage','opp_block_percentage','opp_effective_field_goal_percentage','opp_field_goal_percentage','opp_free_throw_attempt_rate','opp_free_throw_percentage','opp_free_throws_per_field_goal_attempt','opp_offensive_rating','opp_offensive_rebound_percentage','opp_steal_percentage','opp_three_point_attempt_rate','opp_three_point_field_goal_percentage','opp_total_rebound_percentage','opp_true_shooting_percentage','opp_two_point_field_goal_percentage','pace','simple_rating_system','steal_percentage','strength_of_schedule','three_point_attempt_rate','three_point_field_goal_percentage','total_rebound_percentage','true_shooting_percentage','turnover_percentage','two_point_field_goal_percentage']]

This section is where you can input current year games and get an output.

In [168]:
first_four = [['NORTH-CAROLINA-CENTRAL','NORTH-DAKOTA-STATE',16,16],['ST-JOHNS-NY','ARIZONA-STATE',11,11],
              ['BELMONT','TEMPLE',11,11],['FAIRLEIGH-DICKINSON','PRAIRIE-VIEW',16,16]]

first_round = [['DUKE','NORTH-DAKOTA-STATE',1,16],
               ['VIRGINIA-COMMONWEALTH','CENTRAL-FLORIDA',8,9],['MISSISSIPPI-STATE','LIBERTY',5,12],['VIRGINIA-TECH','SAINT-LOUIS',4,13],
               ['MARYLAND','BELMONT',6,11] ,['LOUISIANA-STATE','YALE',3,14],['LOUISVILLE','MINNESOTA',7,10],
               ['MICHIGAN-STATE','BRADLEY',2,15],['GONZAGA','FAIRLEIGH-DICKINSON',1,16],
               ['SYRACUSE','BAYLOR',8,9],['MARQUETTE','MURRAY-STATE',5,12],['FLORIDA-STATE','VERMONT',4,13],
               ['BUFFALO','ARIZONA-STATE',6,11],['TEXAS-TECH','NORTHERN-KENTUCKY',3,14],['NEVADA','FLORIDA',7,10],['MICHIGAN','MONTANA',2,15],
               ['VIRGINIA','GARDNER-WEBB',1,16],['MISSISSIPPI','OKLAHOMA',8,9],['WISCONSIN','OREGON',5,12], 
               ['KANSAS-STATE','CALIFORNIA-IRVINE',4,13],['VILLANOVA','SAINT-MARYS-CA',6,11], ['PURDUE','OLD-DOMINION',3,14],
               ['CINCINNATI','IOWA',7,10],['TENNESSEE','COLGATE',2,15],['NORTH-CAROLINA','IONA',1,16],['UTAH-STATE','WASHINGTON',8,9],
               ['AUBURN','NEW-MEXICO-STATE',5,12],['KANSAS','NORTHEASTERN',4,13],['IOWA-STATE','OHIO-STATE',6,11],['HOUSTON','GEORGIA-STATE',3,14],
               ['WOFFORD','SETON-HALL',7,10],['KENTUCKY','ABILENE-CHRISTIAN',2,15]]

second_round = [['DUKE','CENTRAL-FLORIDA',1,9],['LIBERTY','VIRGINIA-TECH',12,4],['LOUISIANA-STATE','BELMONT',3,11],
                  ['MINNESOTA','MICHIGAN-STATE',10,2],['GONZAGA','BAYLOR',1,9],['MURRAY-STATE','FLORIDA-STATE',12,4],['ARIZONA-STATE','TEXAS-TECH',11,3]
                 ,['FLORIDA','MICHIGAN',10,2],['VIRGINIA','OKLAHOMA',1,9],['OREGON','CALIFORNIA-IRVINE',12,13],['VILLANOVA','PURDUE',6,3],
                 ['CINCINNATI','TENNESSEE',7,2],['NORTH-CAROLINA','WASHINGTON',1,9],['NEW-MEXICO-STATE','KANSAS',12,4],['OHIO-STATE','HOUSTON',11,3],
                 ['SETON-HALL','KENTUCKY',10,2]]

sweet_sixteen = [['DUKE','VIRGINIA-TECH',1,4],['LOUISIANA-STATE','MICHIGAN-STATE',3,2],['GONZAGA','FLORIDA-STATE',1,4],
                  ['TEXAS-TECH','MICHIGAN',3,2],['VIRGINIA','OREGON',1,13],['TENNESSEE','VILLANOVA',2,6],['NORTH-CAROLINA','KANSAS',1,4]
                 ,['OHIO-STATE','KENTUCKY',11,2]]

elite_eight = [['DUKE','MICHIGAN-STATE',1,2],['GONZAGA','MICHIGAN',1,2],['VIRGINIA','VILLANOVA',1,6],
                  ['KANSAS','KENTUCKY',4,2]]

final_four = [['DUKE','MICHIGAN',1,2],['KENTUCKY','VILLANOVA',2,6]]

finals = [['DUKE','VILLANOVA',1,6]]


In [185]:
game_df_cur_team = pd.DataFrame(finals)
game_df_cur_team.columns = ['Away','Home','Away_Seed','Home_Seed']

In [186]:
game_df_cur = pd.DataFrame(finals)
game_df_cur.columns = ['Away','Home','Away_Seed','Home_Seed']
game_df_cur = game_df_cur[['Away','Home']]
game_df_cur = game_df_cur.merge(teams_df_19, left_on = ['Home'], right_on = ['abbreviation'], how = 'left')
game_df_cur = game_df_cur.merge(teams_df_19, left_on = ['Away'], right_on = ['abbreviation'], how = 'left')

In [187]:
game_input_cur = game_df_cur.drop(columns = ['Away','Home','abbreviation_x','abbreviation_y'], axis = 1)
game_input_cur = game_input_cur[['assist_percentage_x','block_percentage_x','effective_field_goal_percentage_x','field_goal_percentage_x','free_throw_attempt_rate_x','free_throw_percentage_x','net_rating_x','offensive_rating_x','offensive_rebound_percentage_x','pace_x','simple_rating_system_x','steal_percentage_x','strength_of_schedule_x','three_point_attempt_rate_x','three_point_field_goal_percentage_x','total_rebound_percentage_x','true_shooting_percentage_x','turnover_percentage_x','two_point_field_goal_percentage_x','assist_percentage_y','block_percentage_y','effective_field_goal_percentage_y','field_goal_percentage_y','free_throw_attempt_rate_y','free_throw_percentage_y','net_rating_y','offensive_rating_y','offensive_rebound_percentage_y','pace_y','simple_rating_system_y','steal_percentage_y','strength_of_schedule_y','three_point_attempt_rate_y','three_point_field_goal_percentage_y','total_rebound_percentage_y','true_shooting_percentage_y','turnover_percentage_y','two_point_field_goal_percentage_y']]

In [188]:
cols = [['assist_percentage_x','block_percentage_x','effective_field_goal_percentage_x','field_goal_percentage_x','free_throw_attempt_rate_x','free_throw_percentage_x','net_rating_x','offensive_rating_x','offensive_rebound_percentage_x','pace_x','simple_rating_system_x','steal_percentage_x','strength_of_schedule_x','three_point_attempt_rate_x','three_point_field_goal_percentage_x','total_rebound_percentage_x','true_shooting_percentage_x','turnover_percentage_x','two_point_field_goal_percentage_x','assist_percentage_y','block_percentage_y','effective_field_goal_percentage_y','field_goal_percentage_y','free_throw_attempt_rate_y','free_throw_percentage_y','net_rating_y','offensive_rating_y','offensive_rebound_percentage_y','pace_y','simple_rating_system_y','steal_percentage_y','strength_of_schedule_y','three_point_attempt_rate_y','three_point_field_goal_percentage_y','total_rebound_percentage_y','true_shooting_percentage_y','turnover_percentage_y','two_point_field_goal_percentage_y']]
for col in cols:
    game_input_cur[col] = scale(game_input_cur[col])

In [189]:
i = 0
n = 1000
games_out = pd.DataFrame()
while i < n:
    X_train, X_test, y_train, y_test = train_test_split(X_all, y_all,
                                                   stratify = y_all)
    
    # TODO: Create the parameters list you wish to tune
    parameters = { 'learning_rate' : [0.1],
                   'n_estimators' : [40],
                   'max_depth': [3],
                   'min_child_weight': [3],
                   'gamma':[0.4],
                   'subsample' : [0.8],
                   'colsample_bytree' : [0.8],
                   'scale_pos_weight' : [1],
                   'reg_alpha':[1e-5]
                 }  

    # TODO: Initialize the classifier
    clf = xgb.XGBClassifier(seed=2)

    # TODO: Make an f1 scoring function using 'make_scorer' 
    f1_scorer = make_scorer(f1_score,pos_label='X')

    # TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
    grid_obj = GridSearchCV(clf,
                            scoring=f1_scorer,
                            param_grid=parameters,
                            cv=5)

    # TODO: Fit the grid search object to the training data and find the optimal parameters
    grid_obj = grid_obj.fit(X_train,y_train)

    # Get the estimator
    clf = grid_obj.best_estimator_
    print(clf)

    # Report the final F1 score for training and testing after parameter tuning
    f1, acc = predict_labels(clf, X_train, y_train)
    print("F1 score and accuracy score for training set:", f1, " , ", acc)

    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set:", f1, " , ", acc)
    
    out = clf.predict(game_input_cur)
    temp = pd.DataFrame(out)
    games_df_out_cur = game_df_cur_team.merge(temp, left_index = True, right_index = True)
    games_df_out_cur.columns = ['Away','Home','Away_Seed','Home_Seed','Forecast']
    games_df_out_cur['Winner'] = np.where(games_df_out_cur['Forecast'] == 'Y', games_df_out_cur['Away'], games_df_out_cur['Home'])
    games_df_out_cur['Wins'] = 1
    games_out_temp = games_df_out_cur[['Winner','Wins']]
    games_out = pd.concat([games_out,games_out_temp])
    i += 1

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.4, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=3, missing=None,
       n_estimators=40, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=1e-05,
       reg_lambda=1, scale_pos_weight=1, seed=2, silent=True,
       subsample=0.8)
Made predictions in 0.0030517578125 seconds.
F1 score and accuracy score for training set: 0.9002320185614849  ,  0.8922305764411027
Made predictions in 0.0010020732879638672 seconds.
F1 score and accuracy score for test set: 0.6950354609929078  ,  0.6791044776119403


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.4, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=3, missing=None,
       n_estimators=40, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=1e-05,
       reg_lambda=1, scale_pos_weight=1, seed=2, silent=True,
       subsample=0.8)
Made predictions in 0.0020058155059814453 seconds.
F1 score and accuracy score for training set: 0.8990825688073395  ,  0.8897243107769424
Made predictions in 0.001003265380859375 seconds.
F1 score and accuracy score for test set: 0.7483870967741936  ,  0.7089552238805971


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.4, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=3, missing=None,
       n_estimators=40, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=1e-05,
       reg_lambda=1, scale_pos_weight=1, seed=2, silent=True,
       subsample=0.8)
Made predictions in 0.003006458282470703 seconds.
F1 score and accuracy score for training set: 0.8747152619589976  ,  0.8621553884711779
Made predictions in 0.0020034313201904297 seconds.
F1 score and accuracy score for test set: 0.7450980392156864  ,  0.7089552238805971


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.4, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=3, missing=None,
       n_estimators=40, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=1e-05,
       reg_lambda=1, scale_pos_weight=1, seed=2, silent=True,
       subsample=0.8)
Made predictions in 0.0025060176849365234 seconds.
F1 score and accuracy score for training set: 0.8796296296296297  ,  0.8696741854636592
Made predictions in 0.00701904296875 seconds.
F1 score and accuracy score for test set: 0.7189542483660132  ,  0.6791044776119403


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

In [190]:
games_prob = games_out.groupby(['Winner']).aggregate(np.sum)
games_prob = games_prob.reset_index()
games_prob['Win %'] = games_prob['Wins'] / n
games_prob = games_prob[['Winner','Win %']]
games_prob

,Winner,Win %
0,DUKE,0.93
1,VILLANOVA,0.07


In [191]:
games_df_out_prob_v1 = game_df_cur_team.merge(games_prob, left_on = 'Away', right_on = 'Winner', how = 'left')
games_df_out_prob = games_df_out_prob_v1.merge(games_prob, left_on = 'Home', right_on = 'Winner', how = 'left')
games_df_out_prob[['Win %_x','Win %_y']] = games_df_out_prob[['Win %_x','Win %_y']].fillna(0.0)
games_df_out_prob['Winner'] = np.where(games_df_out_prob['Win %_x'] > games_df_out_prob['Win %_y'], games_df_out_prob['Winner_x'], games_df_out_prob['Winner_y'])
games_df_out_prob['Win %'] = np.where(games_df_out_prob['Win %_x'] > games_df_out_prob['Win %_y'], games_df_out_prob['Win %_x'], games_df_out_prob['Win %_y'])
games_df_out_prob = games_df_out_prob[['Away','Home','Away_Seed','Home_Seed','Winner','Win %']]

In [192]:
games_df_out_prob

,Away,Home,Away_Seed,Home_Seed,Winner,Win %
0,DUKE,VILLANOVA,1,6,DUKE,0.93


In [ ]:
final_four = [['DUKE','MICHIGAN',1,2],['KENTUCKY','VILLANOVA',2,6]]